# Step 04: Model performance

## A. Lift, gains, Hosmer-Lemeshow, KS stat

In [ ]:
def get_gain_lift_ks(df, ntile, predictedCol, observed):
    df = (df.withColumn('Decile',F.ntile(ntile).over(Window.orderBy(F.desc(predictedCol)))).groupBy(F.col('Decile'))\
         .agg(F.min(F.col(predictedCol)).alias('Min_Prob'),
              F.max(F.col(predictedCol)).alias('Max_Prob'),
              F.mean(F.col(observed)).alias('Response_rate'),
              F.mean(F.col(predictedCol)).alias('Predicted_Response_rate'),
              F.sum(F.col(observed)).alias('Events'),
              F.count(F.lit(1)).alias('Count')
             )
         )
    pdf = df.toPandas()
    
    pdf['NonEvents'] = pdf['Count'] - pdf['Events']
    #events / non-events rate
    pdf['Event_rate'] = pdf[['Events']].apply(lambda x:x/float(x.sum()))
    pdf['NonEvent_rate'] = pdf[['NonEvents']].apply(lambda x:x/float(x.sum()))
    # cumulative event rate / non event rate
    pdf['Cum_event_rate'] = pdf[['Event_rate']].cumsum()
    pdf['Cum_Nonevent_rate'] = pdf[['NonEvent_rate']].cumsum()
    # KS statistic
    pdf['KS'] = round(100*(pdf['Cum_event_rate'] - pdf['Cum_nonevent_rate']),4)
    # MAD
    pdf['|Diff_actual_prediction|'] = abs(df['Response_rate'] - pdf['Predicted_response_rate'])
    gainsLift = pdf
    
    #lift
    responseRate = 100*sum(gainsLift['Events'])/sum(gainsLift['Count'])
    gainsLift['Lift'] = 100*gainsLift[['Response Rate']]/responseRate
    #Cumulative lift
    gainsLift['Cum_response_rate'] = (gainsLift["Events"].cumsum()/gainsLift['Count'].cumsum())
    gainsLift['Cum_Lift'] = 100*gainsLift[['Cum_response_rate']]/responseRate
    #data fraction
    gainsLift['Data_fraction'] = 1.0/ntile
    gainsLift['Cum_data_fraction'] = gainsLift[['Data_fraction']].cumsum()
    
    return gainsLift


def plot_gainLift(df_gainsLift):
    responseRate = 100*sum(df_gainsLift['Events'])/sum(df_gainsLift['Count'])
    fig,(ax1,ax2) = plt.subplots(1,2,figsize=(12,6))
    ax1.plot(100*df_gainsLift['Cum_data_fraction'],100*df_gainsLift['Cum_event_rate'])
    ax1.plot([0,100],[0,100],color='navy',lw=2,linestyle='--')
    ax1.plot([0,responseRate,100],[0,100,100],color='navy',lw=2,linestyle='--')
    ax1.set_xlabel('Percentile')
    ax1.set_ylabel('Gains')
    ax1.set_title('Gains curve')
    ax1.grid(True)
    
    #plot lift
    ax2.plot(100*df_gainsLift['Cum_Data_fraction'],df_gainsLift['Cum_lift'])
    ax2.set_xlabel('Percentile')
    ax2.set_ylabel('Lift')
    ax2.set_title('Lift Curve')
    ax2.grid(True)
    plt.show()
    
    

def get_pct_dev(sdf, predictedCol, observed):
    #pct_dev = (abs(sdf[predictedCol].sum() - sdf[observed].sum())/sdf[observed].sum())
    
    pct_dev_df = sdf.agg( F.abs(F.sum(F.col(predictedCol)) - F.sum(F.col(observed))).alias('abs_res'),
                         F.sum(F.col(observed)).alias('sum_actual'),
                         (F.abs(F.sum(F.col(predictedCol)) - F.sum(F.col(observed)))).alias('pct_dev')).toPandas()
    return pct_dev_df.iloc[0,2]
                               


# Hosmer-Lemeshow   
def HL_table_sdf(sdf,ntile,observed,predictedCol,sig):
    sdf = (sdf.withColumn('Decile', F.ntile(ntile),over(Window.orderBy(F.desc(predictedCol)))).groupBy(F.col('Decile'))\
            .agg(F.min(F.col(predictedCol)).alias("Score_low"),
                 F.max(F.col(predictedCol)).alias("Score_high"),
                 F.count(F.lit(1)).alias("Total Obs"),
                 F.mean(F.col(observed)).alias("Act_Events_Pct"),
                 F.mean(F.col(predictedCol)).alias("Pred_Events_Pct"),
                 F.sum(F.col(observed)).alias("Act_Events")
                )
            )
    
    pdf = sdf.toPandas()
    pdf['Exp_Events'] = (pdf['Total_Obs']*pdf['Pred_Events_Pct']).round()
    pdf['Act_Non_Events'] = pdf['Total_Obs'] - pdf['Act_events']
    pdf['Exp_Non_Events'] = (pdf['Total_Obs']*(1-pdf['Pred_Events_Pct'])).round()
    pdf['HLstat_Events'] = np.square(pdf['Act_Events'] - pdf['Exp_Events'])/pdf['Exp_Events']
    pdf['HLstat_NonEvents'] = np.square(pdf['Act_Non_Events'] - pdf['Exp_Non_Events'])/pdf['Exp_Non_Events']
    pdf['HL_stat'] = pdf['HLStat_Events'] + pdf['HLStat_NonEvents']
    
    #check HL test significant
    HL_stat = pdf['HL_stat'].sum()
    d_f = pdf['Decile'].max()-2
    p_value = 1-chi2.cdf(HL_stat,d_f)
    
    #signficant level
    print("Hosmer-Lemeshow test statistics is {} and p-values is {} at degree of freedom {}".format(round(HL_stat,2),
                                                                                                   p_value,
                                                                                                   d_f))
    if p_value < sig:
        print("HL test is significant at level {}, which indicates that the model is not a good fit".format(sig))
    else:
        print("HL test is not significant at level {}, which indicates that the model is a good fit".format(sig))
        
    #generate tables
    df_out = pdf.to_html(formatters = {
                        'Decile':'{:,.0f}'.format,
                        'Total_Obs':'{:,.0f}'.format,
                        'Act_events':'{:,.0f}'.format,
                        'Act_Non_events':'{:,.0f}'.format,
                        'Exp_events':'{:,.0f}'.format,
                        'Exp_Non_events':'{:,.0f}'.format,
                        'Act_Events_Pct':'{:,.2%}'.format,
                        'Pred_Events_Pct':'{:,.2%}'.format})
    display(HTML(df_out))
    
    

# pozzolo formula
def prob_adj_pozzolo(df, beta, p_col):
    p_s = df[p_col]
    p_adj = (p_s * beta) / (p_s * beta - p_s + 1)
    df['P_adj'] = p_adj
    return df


# runs HL, deciles
def generate_gains_lift_HL(sdf, path, name, target, orig_distr):
    #calibrate both in-time/out-time validation since they reflect true distribition 
    #because training was downsampled, thus leading to higher probability than true distr
    if orig_distr:
        predictedCol = 'p_adj'
    else:
        predictedCol = 'p1'
        
    #Hosmer-Lemeshow
    HL_df = HL_table_sdf(sdf = sdf,
                        ntile = 10,
                        observed = target,
                        predictedCol = predictedCol,
                        sig = 0.05)
    
    print("******************** KS *********************")
    gain_lift = get_gain_lift_ks(sdf, 10, predictedCol, target)
    pct_dev = get_pct_dev(sdf, predictedCol, target)
    print("Difference between predicted and actuals is {:0.7f}".format(pct_dev))
    
    plot_gainLift(gain_lift)
    gain_lift.to_csv(path + name + '_KSchart.csv')

## Decile chart

In [ ]:
def decile_charts_train(path, scrn_data, name, clf_name):
    
    #read spark dataframe using raw prob "p1" because we use down-sampled data
    df = (scrn_data.withColumn('Decile',F.ntile(10).over(Window.orderBy(F.desc('p1')))).groupBy(F.col('Decile'))\
        .agg(F.min(F.col('p1')).alias('Min_Prob'),  #praw
            F.max(F.col('p1')).alias('Max_Prob'),
            F.mean(F.col('target')).alias('Actual'),
            F.mean(F.col('p1')).alias('Raw_Prob'),
            F.sum(F.col('target')).alias('Events'),
            F.count(F.lit(1)).alias('Min_Prob'))
        )
    pdf = df.toPandas()
    
    #place max probability into dataframe
    raw_pred = pd.DataFrame(pdf['max_probability'])
    
    # ---------------------------------------
    # save cutoffs from training set
    # ---------------------------------------
    interval_lst = raw_pred['max_prob'].values.tolist()
    interval_lst.sort(reverse=False)
    interval_lst.insert(0, -float('inf'))
    interval_lst.pop()
    interval_lst.append(float('inf'))
    
    #pickle file
    filename = (path + clf_name + '_train_decile_cutoffs')
    pickle.dump(interval_lst, open(filename,'wb'))
    print('Cutoffs saved to pickle')
    
    # ===================================
    # print final table 
    # ===================================
    pdf.to_csv(path + name + '_decile_tbl.csv')
    print('table hg been saved to CSV')
    
    # ===================================
    # plot figure
    # ===================================
    plt.figure(figsize=(10,6))
    plt.plot(np.array(pdf['Decile']),np.array(pdf['Actual']),marker='.',marketsize=10,label='Actual Response Rate')
    plt.plot(np.array(pdf['Decile']),np.array(pdf['RawProb']),marker='.',marketsize=10,label='Raw Probability')
    plt.xlabel('Decile')
    plt.ylabel('Mean Probability')
    plt.legend(loc='best')
    plt.title('Plot of mean probabilities')
    plt.show()
    


def decile_charts_val(path, scrn_data, name, clf_name):
    decile_map = {0:10,
                     1:9,
                     2:8,
                     3:7,
                     4:6,
                     5:5,
                     6:4,
                     7:3,
                     8:2,
                     9:1}
        
    # =============================================
    # apply decile cutoffs on validation sets
    # =============================================
    filename = (path + clf_name + '_train_decile_cutoffs')
    cutoffs = pickle.load(open(filename,'rb'))
    print('Cutoffs has been loaded')
    
    #read spark dataframe using raw "p1"
    bucketizer = Bucketizer(splits = cutoffs, inputCol='p1', outputCol = 'buckets')  #p_adj is adjusted, praw 
    scrn_data_n = bucketizer.setHandleInvalid('keep').transform(scrn_data)
    
    df = scrn_data_n.groupBy('buckets')\
                    .agg(F.mean(F.col('target')).alias('actual'),
                         F.sum(F.col('target')).alias('tot events'),
                         F.mean(F.col('p_adj')).alias('RawProb'),   #reporting using calibrated P
                         F.min(F.col('p_adj')).alias('min_predict_prob'),
                         F.max(F.col('p_adj')).alias('max_predict_prob'),
                         F.count(F.lit(1)).alias('Count'))
    pdf = df.toPandas()
    pdf = pdf.sort_values(by = 'buckets', ascending = False)   #bucket #9 represents first decile
    pdf['buckets'] = pdf['buckets'].map(decile_map)
    
    
    # =============================================
    # print final table
    # =============================================
    pdf.to_csv(path + name + '_decile_tbl.csv')
    print('table has been saved to CSV')
    
    
    # =============================================
    # plot figure
    # =============================================
    plt.figure(figsize=(10,6))
    plt.plot(np.array(pdf['buckets']),np.array(pdf['Actual']),marker='.',marketsize=10,label='Actual Response Rate')
    plt.plot(np.array(pdf['buckets']),np.array(pdf['RawProb']),marker='.',marketsize=10,label='Raw Probability')
    plt.xlabel('Decile')
    plt.ylabel('Mean Probability')
    plt.legend(loc='best')
    plt.title('Plot of mean probabilities')
    plt.show()

## General performance metrics

In [ ]:
def to_labels(pos_probs, threshold):
    return (pos_probs > threshold).astype('int')

def calc_model_perf_train(name, X, smpl, pcol):
    #declare new dict to store results
    model_perf = {}
    model_perf['smpl'] = smpl
    model_perf['model'] = name
    model_perf['brier_score'] = brier_score_loss(X['target'],X[pcol])
    model_perf['log_loss'] = log_loss(X['target'],X[pcol])
    model_perf['ROC'] = roc_auc_score(X['target'],X[pcol])
    
    #calculate precision and recall values
    precision, recall, thresholds = precision_recall_curve(X['target'].values, x[pcol].values)
    model_perf=['AUC using precision-recall values'] = auc(recall,precision)
    
    # ===========================================
    # Option 0: using default 50% threshold 
    # ===========================================
    model_perf['tot_pred_class_default'] = X['pclass'].sum()
    model_perf['accuracy_default'] = accuracy_score(X['target'],X['pclass'])
    model_perf['precision_default'] = precision_score(X['target'],X['pclass'])
    model_perf['recall_default'] = recall_score(X['target'],X['pclass'])
    model_perf['f1_score_default'] = f1_score(X['target'],X['pclass'])
    
    tbl = pd.crosstab(X['pclass'],X['target'],margins=True)
    model_perf['TPR_class_default'] = (tbl.iloc[1,1] / tbl.iloc[2,1])
    model_perf['FPR_class_default'] = (tbl.iloc[1,0] / tbl.iloc[2,0])
    model_perf['FNR_class_default'] = (tbl.iloc[0,1] / tbl.iloc[2,1])
    model_perf['TNR_class_default'] = (tbl.iloc[0,0] / tbl.iloc[2,0])
    model_perf['PPV_class_default'] = (tbl.iloc[1,1] / tbl.iloc[1,2])
    
    
    # ========================================================
    # Option 1: using ROC curve to find optimal threshold
    # ========================================================
    fpr, tpr, thresholds = roc_curve(X['target'].values, X[pcol].values)
    
    #get best threshold
    J = tpr - fpr
    ix = np.argmax(J)
    best_thresh = thresholds[ix]
    print("Best threshold using ROC curve=%f" %(best_thresh))
    
    #optimize class using best thresholds
    X['class_roc'] = np.where(X[pcol] > best_thresh,1,0)
    
    model_perf['tot_pred_class_roc'] = X['class_roc'].sum()
    model_perf['accuracy_class_roc'] = accuracy_score(X['target'],X['class_roc'])
    model_perf['precision_class_roc'] = precision_score(X['target'],X['class_roc'])
    model_perf['recall_class_roc'] = recall_score(X['target'],X['class_roc'])
    model_perf['f1_score_class_roc'] = f1_score(X['target'],X['class_roc'])
    
    tbl = pd.crosstab(X['class_roc'],X['target'],margins=True)
    model_perf['TPR_class_roc'] = (tbl.iloc[1,1] / tbl.iloc[2,1])
    model_perf['FPR_class_roc'] = (tbl.iloc[1,0] / tbl.iloc[2,0])
    model_perf['FNR_class_roc'] = (tbl.iloc[0,1] / tbl.iloc[2,1])
    model_perf['TNR_class_roc'] = (tbl.iloc[0,0] / tbl.iloc[2,0])
    model_perf['PPV_class_roc'] = (tbl.iloc[1,1] / tbl.iloc[1,2])
    
    
    # =====================================================================
    # Option 2: using precision-recall curve to find optimal threshold
    # =====================================================================
    #calculate pr curve
    precision, recall, thresholds = precision_recall_curve(X['target'].values, X[pcol].values)
    fscore = (2 * precision * recall)/(precision + recall)
    
    #located index of largest f-score
    ix = np.argmax(fscore)
    print("best threshold using precision-recall=%f, F-score=%0.3f" %(thresholds[ix],fscore[ix]))
    
    #optimize class using best threshold
    X['class_pr'] = np.where(X[pcol] > thresholds[ix],1,0)
    
    model_perf['tot_pred_class_pr'] = X['class_pr'].sum()
    model_perf['accuracy_class_pr'] = accuracy_score(X['target'],X['class_pr'])
    model_perf['precision_class_pr'] = precision_score(X['target'],X['class_pr'])
    model_perf['recall_class_pr'] = recall_score(X['target'],X['class_pr'])
    model_perf['f1_score_class_pr'] = f1_score(X['target'],X['class_pr'])
    
    tbl = pd.crosstab(X['class_pr'],X['target'],margins=True)
    model_perf['TPR_class_pr'] = (tbl.iloc[1,1] / tbl.iloc[2,1])
    model_perf['FPR_class_pr'] = (tbl.iloc[1,0] / tbl.iloc[2,0])
    model_perf['FNR_class_pr'] = (tbl.iloc[0,1] / tbl.iloc[2,1])
    model_perf['TNR_class_pr'] = (tbl.iloc[0,0] / tbl.iloc[2,0])
    model_perf['PPV_class_pr'] = (tbl.iloc[1,1] / tbl.iloc[1,2])
    
    
    # ===========================================================================
    # Option 3: using precision-recall curve to find optimal threshold (tuning)
    # ===========================================================================
    #define thresholds
    thresholds2 = np.arange(0,1,0.001)
    
    #evaluate best threshold
    scores = [f1_score(X['target'].values, to_labels(X[pcol].values,t)) for t in thresholds2]
    
    #get best tresholds
    ix = np.argmax(scores)
    print("Best threshold using precision-recall=%f, F-score=%0.3f" % (threhsolds2[ix],scores[ix]))
    
    #optimize class using best threshold
    X['class_pr_tuning'] = np.where(X[pcol] > thresholds2[ix],1,0)
    
    model_perf['tot_pred_class_pr_tuning'] = X['class_pr_tuning'].sum()
    model_perf['accuracy_class_pr_tuning'] = accuracy_score(X['target'],X['class_pr_tuning'])
    model_perf['precision_class_pr_tuning'] = precision_score(X['target'],X['class_pr_tuning'])
    model_perf['recall_class_pr_tuning'] = recall_score(X['target'],X['class_pr_tuning'])
    model_perf['f1_score_class_pr_tuning'] = f1_score(X['target'],X['class_pr_tuning'])
    
    tbl = pd.crosstab(X['class_pr_tuning'],X['target'],margins=True)
    model_perf['TPR_class_pr_tuning'] = (tbl.iloc[1,1] / tbl.iloc[2,1])
    model_perf['FPR_class_pr_tuning'] = (tbl.iloc[1,0] / tbl.iloc[2,0])
    model_perf['FNR_class_pr_tuning'] = (tbl.iloc[0,1] / tbl.iloc[2,1])
    model_perf['TNR_class_pr_tuning'] = (tbl.iloc[0,0] / tbl.iloc[2,0])
    model_perf['PPV_class_pr_tuning'] = (tbl.iloc[1,1] / tbl.iloc[1,2])
    
    #prepare final dataframe
    return pd.DataFrame.from_dict(model_perf, orient = 'index', columns=['metrics'])



# ===========================================================================
# Using this on validation - threshold from training
# ===========================================================================
def calc_model_perf_val(name, X, smpl, roc_thresh, pr_thresh, pr_tuning_thresh, pcol):
    
    #declare new dict to store results
    model_perf = {}
    model_perf['smpl'] = smpl
    model_perf['model'] = name
    model_perf['brier_score'] = brier_score_loss(X['target'],X[pcol])
    model_perf['log_loss'] = log_loss(X['target'],X[pcol])
    model_perf['ROC'] = roc_auc_score(X['target'],X[pcol])
    
    #calculate precision and recall values
    precision, recall, thresholds = precision_recall_curve(X['target'].values, x[pcol].values)
    model_perf=['AUC using precision-recall values'] = auc(recall,precision)
    
    # ===========================================
    # Option 0: using default 50% threshold 
    # ===========================================
    model_perf['tot_pred_class_default'] = X['pclass'].sum()
    model_perf['accuracy_default'] = accuracy_score(X['target'],X['pclass'])
    model_perf['precision_default'] = precision_score(X['target'],X['pclass'])
    model_perf['recall_default'] = recall_score(X['target'],X['pclass'])
    model_perf['f1_score_default'] = f1_score(X['target'],X['pclass'])
    
    tbl = pd.crosstab(X['pclass'],X['target'],margins=True)
    model_perf['TPR_class_default'] = (tbl.iloc[1,1] / tbl.iloc[2,1])
    model_perf['FPR_class_default'] = (tbl.iloc[1,0] / tbl.iloc[2,0])
    model_perf['FNR_class_default'] = (tbl.iloc[0,1] / tbl.iloc[2,1])
    model_perf['TNR_class_default'] = (tbl.iloc[0,0] / tbl.iloc[2,0])
    model_perf['PPV_class_default'] = (tbl.iloc[1,1] / tbl.iloc[1,2])
    
    
    # ========================================================
    # Option 1: using ROC curve to find optimal threshold
    # ========================================================
    #optimize class using best thresholds
    X['class_roc'] = np.where(X[pcol] > roc_thresh,1,0)
    
    model_perf['tot_pred_class_roc'] = X['class_roc'].sum()
    model_perf['accuracy_class_roc'] = accuracy_score(X['target'],X['class_roc'])
    model_perf['precision_class_roc'] = precision_score(X['target'],X['class_roc'])
    model_perf['recall_class_roc'] = recall_score(X['target'],X['class_roc'])
    model_perf['f1_score_class_roc'] = f1_score(X['target'],X['class_roc'])
    
    tbl = pd.crosstab(X['class_roc'],X['target'],margins=True)
    model_perf['TPR_class_roc'] = (tbl.iloc[1,1] / tbl.iloc[2,1])
    model_perf['FPR_class_roc'] = (tbl.iloc[1,0] / tbl.iloc[2,0])
    model_perf['FNR_class_roc'] = (tbl.iloc[0,1] / tbl.iloc[2,1])
    model_perf['TNR_class_roc'] = (tbl.iloc[0,0] / tbl.iloc[2,0])
    model_perf['PPV_class_roc'] = (tbl.iloc[1,1] / tbl.iloc[1,2])
    
    
    # =====================================================================
    # Option 2: using precision-recall curve to find optimal threshold
    # =====================================================================
    #optimize class using best threshold
    X['class_pr'] = np.where(X[pcol] > pr_thresh,1,0)
    
    model_perf['tot_pred_class_pr'] = X['class_pr'].sum()
    model_perf['accuracy_class_pr'] = accuracy_score(X['target'],X['class_pr'])
    model_perf['precision_class_pr'] = precision_score(X['target'],X['class_pr'])
    model_perf['recall_class_pr'] = recall_score(X['target'],X['class_pr'])
    model_perf['f1_score_class_pr'] = f1_score(X['target'],X['class_pr'])
    
    tbl = pd.crosstab(X['class_pr'],X['target'],margins=True)
    model_perf['TPR_class_pr'] = (tbl.iloc[1,1] / tbl.iloc[2,1])
    model_perf['FPR_class_pr'] = (tbl.iloc[1,0] / tbl.iloc[2,0])
    model_perf['FNR_class_pr'] = (tbl.iloc[0,1] / tbl.iloc[2,1])
    model_perf['TNR_class_pr'] = (tbl.iloc[0,0] / tbl.iloc[2,0])
    model_perf['PPV_class_pr'] = (tbl.iloc[1,1] / tbl.iloc[1,2])
    
    # ===========================================================================
    # Option 3: using precision-recall curve to find optimal threshold (tuning)
    # ===========================================================================
    #optimize class using best threshold
    X['class_pr_tuning'] = np.where(X[pcol] > pr_tuning_thresh,1,0)
    
    model_perf['tot_pred_class_pr_tuning'] = X['class_pr_tuning'].sum()
    model_perf['accuracy_class_pr_tuning'] = accuracy_score(X['target'],X['class_pr_tuning'])
    model_perf['precision_class_pr_tuning'] = precision_score(X['target'],X['class_pr_tuning'])
    model_perf['recall_class_pr_tuning'] = recall_score(X['target'],X['class_pr_tuning'])
    model_perf['f1_score_class_pr_tuning'] = f1_score(X['target'],X['class_pr_tuning'])
    
    tbl = pd.crosstab(X['class_pr_tuning'],X['target'],margins=True)
    model_perf['TPR_class_pr_tuning'] = (tbl.iloc[1,1] / tbl.iloc[2,1])
    model_perf['FPR_class_pr_tuning'] = (tbl.iloc[1,0] / tbl.iloc[2,0])
    model_perf['FNR_class_pr_tuning'] = (tbl.iloc[0,1] / tbl.iloc[2,1])
    model_perf['TNR_class_pr_tuning'] = (tbl.iloc[0,0] / tbl.iloc[2,0])
    model_perf['PPV_class_pr_tuning'] = (tbl.iloc[1,1] / tbl.iloc[1,2])
    
    #prepare final dataframe
    return pd.DataFrame.from_dict(model_perf, orient = 'index', columns=['metrics'])

## Launch H2o cluster

In [ ]:
# pysparkling

## Load models from H2o

In [ ]:
# ---------------------------
# GBM
# ---------------------------
name = 'gbm_sel'
n_set = 'gbm'

model_path = '/mapr/datalake/gbm_grid2_model.zip'
imported_model = h2o.upload_mojo(model_path)
print("model imported")

## Read scoring files

In [ ]:
# ------------------------------------
# training downsampled
# ------------------------------------
trn_sdf_orig = spark.read.parquet(path + 'train_feat_se_r2.parquet')
print("number of obs:", trn_sdf_orig.count())
print("number of cols:", len(trn_sdf_orig.columns))

# ------------------------------------
# testing / val
# ------------------------------------

In [ ]:
# list of primary key (dates, claim, prov_id)
pk_lst = ['claim_nbr']

# list of numeric var that must be treated as factor in H2o
binary_lst = ['hosp_ind']

print("*****************************")
print("Before removing PK")
print("*****************************")
# ---------------------------
# numeric
# ---------------------------
num_cols = []
for item in trn_sdf.dtypes:
    if item[1].startswith('int') | item[1].startswith('double') | item[1].startswith('bigint'):
        num_cols.append(item[0])
        
print("Numeric list:", num_cols)
print("there are", len(num_cols), "numeric attributes, including target")
print()


# ---------------------------
# categorical
# ---------------------------
cat_cols = []
for item in trn_sdf.dtypes:
    if item[1].startswith('string'):
        cat_cols.append(item[0])
        
print("Categorical list:", cat_cols)
print("there are", len(cat_cols), "categorical attributes, including target")
print() 



#remove PK list
print("*****************************")
print("After removing PK")
print("*****************************")
num_lst_n = []
for i in num_cols:
    if i not in pk_lst and i not in binary_lst:
        num_lst_n.append(i)
        
print("numeric list:", num_lst_n)
print('there are', len(num_lst_n), 'numeric attributes, including target')


cat_lst_n = []
for i in cat_cols:
    if i not in pk_lst:
        cat_lst_n.append(i)
        
# add binary variables into categorical
cat_lst_n.extend(binary_lst)
print("Categorical list:", cat_lst_n)
print("there are", len(cat_lst_n), "categorical attributes, including target")
print()



print("*****************************")
print("Final list")
print("*****************************")
# merge into one list
lst = num_lst_n + cat_lst_n
print("Final list:", lst)
print("there are", len(lst), "categorical attributes, including target")

## Transform into h2o frames

In [ ]:
y = 'target'

# --------------------
# in-time training
# --------------------
print("preparing training")
#trn_sdf_n = trn_sdf.select(*lst)  #only required cols
train_h2o = hc.asH2OFrame(trn_sdf_orig)

#prepare final list of attribres - converting categorical as factor
for i in cat_lst_n:
    train_h2o[i] = train_h2o[i].asfactor()
    
# validation and OOT

In [ ]:
train_h2o.head(5)

## Score datasets and calibrate raw probability

In [ ]:
beta = 0.0005432

# -------------------------------------
# training
# -------------------------------------
print("score training")
pred_trn = imported_model.predict(train_h2o)
pred_trn = pred_trn.drop(['p0'])
# concatenate to original dataset
trn_h2o = train_h2o.cbind(pred_trn)
trn_sdf_n = hc.asSparkFrame(trn_h2o)
trn_sdf_n = trn_sdf_n.withColumnRenamed('call','target')
trn_sdf_n = trn_sdf_n.withColumnRenamed('predict','pclass')
trn_sdf_n = trn_sdf_n.withColumn('target', F.col('target').cast('int'))
trn_sdf_n = trn_sdf_n.withColumn('p_adj', ( F.col('p1')*F.lit(beta) / (F.col('p1')*F.lit(beta) - F.col('p1') + F.lit(1)) ))

# ------------------------
# stats
# ------------------------
print("************** training ****************")
trn_sdf_n.agg(F.sum(F.col('p1')).alias('total predicted target: raw prob'),
              F.sum(F.col(['p_adj'])).alias('total predicted target: adj prob'),
              (F.sum(F.col('target'))/F.countDistrinct(F.col('claim_nbr'))).alias('call rate'),
              F.sum(F.col('target')).alias('total actual target'),
              F.countDistinct(F.col('claim_nbr')).alias('total claims')).show()

## dataframes for model performance

In [ ]:
#training
trn_df = trn_sdf_n.select('target','pclass','p1').toPandas()
trn_df['pclass'] = trn_df['pclass'].astype('int')

print("training:", trn_df.shape)

## model performance

In [ ]:
# ----------------------------------
# in-time training
# ----------------------------------
model_perf_train = {}
model_train = imported_model.model_performance(train_h2o)
model_perf_train['model'] = n_set
model_perf_train['set'] = 'in-time training'
model_perf_train['KS'] = imported_model.kolmogorov_smirnov()
model_perf_train['AUC'] = model_train.auc()
model_perf_train['AUCPR'] = model_train.aucpr()
model_perf_train['Gini'] = model_train.gini()
model_perf_train['LogLoss'] = model_train.logloss()
model_perf_train['Precision'] = model_train.precision()
model_perf_train['Recall'] = model_train.recall()
model_perf_train['F1'] = model_train.F1()
model_perf_train['Accuracy'] = model_train.accuracy()
model_perf_train['best_thresh_F1'] = model_train.find_threshold_by_max_metric("f1")

#prepare table of metrics
perf_df = pd.concat([pd.DataFrame(model_perf_train),
                     pd.DataFrame(model_perf_test),
                     pd.DataFrame(model_perf_oot)],axis=0,sort=False)
perf_df

### =========================================================
### using scikit learn 
### =========================================================

In [ ]:
#score uncalibrated model
saved_clf = load_model(path = path, name = 'xgboost')
pred_trn = pd.concat([pd.DataFrame(saved_clf.predict_prob(train[lst].values)[:,1],columns=['praw']),
                    pd.DataFrame(saved_clf.predict(train[lst].values),columns=['pclass'])],axis=1)
print("training predictions complete")


#score calibrated model
saved_clf_cal = load_model(path = path, name = 'xgboost_platt')
cal_pred_trn = pd.DataFrame(saved_clf_cal.predict(pred_trn['praw']),columns=['pcal'])
print("training predictions complete")

#merge into one data
train_df = pd.concat([train,pred_trn,cal_pred_trn],axis=1)

#select required fields
trn_df = trn_sdf_n.select(['target','praw','pcal','pclass']).toPandas()

### Stats 

In [ ]:
trn_sdf_n.agg(F.sum(F.col('praw')).alias('total predicted target: raw prob'),
             F.sum(F.col('pcal')).alias('total predicted target: adj prob'),
              (F.sum(F.col('target'))/F.countDistinct(F.col('id'))).alias('target rate'),
              F.sum(F.col('target')).alias('total actual target'),
              F.countDistinct(F.col('id')).alias('total mbrs')).show()

# A. Lift, gains, HL stats

## Model 1 GBM

In [ ]:
print("*********** in-time training ***************")
generate_gains_lift_HL(sdf = trn_sdf_n,
                      path = path,
                      name = n_set + '_train',
                      target = 'target',
                      orig_distr = False)   #downsampled


print("*********** in-time testing ***************")
generate_gains_lift_HL(sdf = tst_sdf_n,
                      path = path,
                      name = n_set + '_test',
                      target = 'target',
                      orig_distr = True)   #true distr

In [ ]:
trn_lift = imported_model.gains_lift().as_data_frame()
trn_lift

# B. Decile charts

## Model 1: GBM

In [ ]:
print("*********** in-time training ***************")
decile_charts_train(path = path,
                   scrn_data = trn_sdf_n,
                   name = n_set + '_train',
                   clf_name = n_set)

# C. Explainable AI

## A. Feature importance on training

### Model 01: GBM

In [ ]:
num_lst_n

In [ ]:
%matplotlib inline
exm = imported_model.explain(train_h2o,
                            columns = num_lst_n[:8],
                            exclude_explanations = ['model_correlaiton_heatmap','shap_summary','ice'])

In [ ]:
#imported_model.varimp_plot()
varimp = imported_moedl.varimp(use_pandas = True)
varimp.to_csv(filepath + n_set + 'featImp.csv')
varimp.head(20)

## B. Performance metrics

In [ ]:
print(" *********************** In-Time Training *********************")
perf_df = calc_model_perf_train(name, trn_df, smpl = n_set + '_train', pcol = 'p1')
df1 = perf_df.T

In [ ]:
roc_thresh = 0.18432
pr_thresh = 0.342434
pr_tuning = 0.3424

print("********* IN TIME TESTING ***************")
perf_df = calc_model_perf_train(name, tst_df, smpl = n_set + '_test', 
                                roc_thresh = roc_thresh,
                                pr_thresh = pr_thresh,
                                pr_tuning = pr_tuning,
                                pcol = 'p_adj')
df2 = perf_df.T

#print final table
perf_final = pd.concat([df1,df2,df3],axis=0)
perf_final.to_csv(filepath + n_set + '_modelPerf.csv')
perf_final

## C. PDP plots

In [ ]:
num_lst_n

In [ ]:
col_lst = []
for col in num_lst_n:
    col_lst.append(col)
    imported_model.partial_plot(data = oot_h2o,
                               cols = col_lst,
                               server=True,
                               plot=True,
                               figsize=(6,4),
                               nbins=20)
    cols_lst.remove(col)

In [ ]:
#pd_plot = imported_model.pd_plot(oot_h2o, 'tot_clm')

In [ ]:
imported_model.partial_plot(data = oot_h2o, cols = ['clm_lst6'], server = True, plot = True, figsize=(8,6),
                                                   nbins = 20)

## D. Feature permutation importance

In [ ]:
# H2o
imported_model.permutation_importance_plot(oot_h2o)

# scikitlearn
from sklearn.inspection import permutation_importance
train_result = permutation_importance(
    saved_clf, X, y, n_repeats=10, random_state=42, n_jobs=2
)

sorted_importances_idx = train_result.importances_mean.argsort()

#prepare data for figure
train_importances = pd.DataFrame(
    train_result.importances[sorted_importances_idx].T,
    columns = X.columns[sorted_importances_idx],
)

In [ ]:
plt.rcParams['figure.figsize'] = [10,6]
plt.rcParams['figure.autolayout'] = True

for name, importances in zip(['training','in-time validation','out-time validation'],
                            [train_importances, test_importances, oot_importances]):
    ax = importances.plot.box(vert=False, whis=10)
    ax.set_title(f"Permutation Importances ({name} set)")
    ax.set_xlabel("Decrease in accuracy score")
    ax.axvline(x=0, color = 'k', linestyle = '--')
    ax.figure.tight_layout()

## E. Find optimal cutoffs values
https://medium.com/swlh/determining-a-cut-off-or-threshold-when-working-with-a-binary-dependent-target-variable-7c2342cf2a7c

In [ ]:
#code fragment to add to existing

#true positive rate (1-FNR)
tips['TPR'] = 1 - (1-tips['SENSITIVITY'])
#true negative rate (1 - TPR)
tips['TNR'] = 1 - (1-tips['SPECIFICITY'])

In [ ]:
tbl = pd.crosstab(train_df['pclass'], train_df['target'], margins = True)
print("TPR class pr", (tbl.iloc[1,1]/tbl.iloc[2,1]))
print("FPR class pr", (tbl.iloc[1,0]/tbl.iloc[2,0]))
print("FNR class pr", (tbl.iloc[0,1]/tbl.iloc[2,1]))
tbl

In [ ]:
pd.crosstab(train_df['pclass'], train_df['target']).apply(lambda r:r/r.sum(),axis=1)